In [ ]:
# Reference : https://ampl.com/api/extra/python_quickstart.html

## Step 1: Path Setting

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import random
from operator import add

In [2]:
from bokeh.layouts import row
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [3]:
from amplpy import AMPL, Environment, DataFrame

## Step 2: Create an AMPL object

In [4]:
ampl = AMPL(Environment('C:\\Users\\user\\Desktop\\OR_Kung\\AMPL'))

## Step 3: Select the solver

In [5]:
ampl.setOption('solver','C:\\Users\\user\\Desktop\\OR_Kung\\AMPL\\cplex')  #'gurobi'

## Step 4: Define the model

In [6]:
# ampl.read('FairCHBF.mod')

In [7]:
ampl.eval('''
set I;
set J;

param B {J} >=0;   # benefit of doing the job
param C {J} >=0;   # cost of doing the job
param K {I} >=0;   # capacity of machine 

var X {I,J} >=0 binary;   # Decision var 1

maximize Benefit: sum {i in I, j in J} B[j] * X[i,j];

s.t. Capacity {i in I        }: sum {j in J} C[j] * X[i,j] <= K[i];
s.t. ProdTime {        j in J}: sum {i in I}        X[i,j] <= 1;
s.t. Arrange  {i in I, j in J}:                     X[i,j] >= 0;

''')

## Step 5: Define the initial data

In [96]:
# batch 要做成:
# [ num_machine, num_job, cost, benefit, capacity ]

it_n = 100

def batch (num_machine, num_job, relation_bc, capacity):
    
    m = num_machine
    n = num_job
    
    list_c1 = []    # 內含 100組 n個 job的 cost 組合
    list_b1 = []    # 內含 100組 n個 job的 bnft 組合
    list_k1 = []    # 內含 100組 由 n個 job 加總算出的 k
    
    for i in range(it_n):
        
        list_c2 = []   # 內含 n個 job的 cost
        list_b2 = []   # 內含 n個 job的 bnft
        
        for j in range(n):
            c = int(random.randint(0,50))
            list_c2.append(c)

            if   relation_bc == 'L':
                b = c

            elif relation_bc == 'X':
                b = c**2

            elif relation_bc == 'A':
                b = c**(1/2)

            elif relation_bc == 'R':
                b = random.randint(0,50)  
            list_b2.append(b)

        if   capacity =='N':
            k = 10000 # no capacity

        elif capacity =='L':
            k = sum(list_c2) / m

        elif capacity =='T':
            k = sum(list_c2) * (0.75) / m
    
        list_c1.append(list_c2)
        list_b1.append(list_b2)
        list_k1.append(k)
    
    return  m, n, list_c1, list_b1, list_k1

In [9]:
# Test OK! 不要在正式 run 100組時  print，會很慢 >"<

instance = batch( 5, 20, 'A', 'T' )
print(instance[2])
print(instance[3])


[[33, 27, 40, 46, 22, 14, 36, 48, 16, 39, 50, 35, 10, 50, 42, 13, 27, 38, 32, 49]]
[[5.744562646538029, 5.196152422706632, 6.324555320336759, 6.782329983125268, 4.69041575982343, 3.7416573867739413, 6.0, 6.928203230275509, 4.0, 6.244997998398398, 7.0710678118654755, 5.916079783099616, 3.1622776601683795, 7.0710678118654755, 6.48074069840786, 3.605551275463989, 5.196152422706632, 6.164414002968976, 5.656854249492381, 7.0]]


In [95]:
machine_job  = [(5,20)]
capacity     = ['L','T','N'] 
relation_bc  = ['L','A','X','R'] 

## Step 6: Model Build-up & Solution

In [97]:
# 迭代 instance

TotalBenefit_ijk_sn_1 = []
TotalBenefit_ijk_sn_2 = []

for i in machine_job:
    
    num_machine = i[0]
    num_job     = i[1]

    # 印出編號，但從1開始算而不是從0，再讓 [1,2,3] -> [M1,M2,M3]

    machine1 = ['M']*num_machine
    job1     = ['J']*num_job
    machine2 = map(add, list(range(num_machine)), [1]*num_machine)
    job2     = map(add, list(range(num_job))    , [1]*num_job)
    machine2 = list( str(x) for x in machine2 )
    job2     = list( str(x) for x in job2     )
    machine  = list (map(lambda y, z: y + z, machine1, machine2))
    job      = list (map(lambda y, z: y + z, job1,     job2    ))

    # AMPL sets

    ampl.getSet('I').setValues(machine)
    ampl.getSet('J').setValues(job)
    
    for k in capacity:    
        for j in relation_bc:

            instance = batch( i[0], i[1], j, k )
            #print(instance)  
            
            TotalBenefit_1 = []
            TotalBenefit_2 = []
            
            for p in range(it_n):      
                
                # ====== Model 1 ======= : 一組一組用solver算 IP (or linear relaxed LP)，每種senario要算 100組 再平均
                
                # amplpy DataFrame
                ampl.setData(DataFrame(
                index=[('J', job)], 
                columns=[
                    ('B', instance[3][p]), 
                    ('C', instance[2][p])
                ]
                ))

                # Pandas DataFrame
                df = pd.DataFrame({
                    'K': [ instance[4][p] ]*num_machine
                }, 
                    index = machine
                )
                ampl.setData(DataFrame.fromPandas(df))

                # Solve the problem
                ampl.solve()

                '''
                # Create a DataFrame with Decision variables
                Var = ampl.getVariable('X').getValues().toPandas()
                print(Var)
                '''

                # Display the objective value
                TotalBnft_1 = ampl.getObjective('Benefit')
                TotalBnft_1 = TotalBnft_1.value()
                TotalBenefit_1 = np.append(TotalBenefit_1, TotalBnft_1) # numpy.ndarray
                
                
                # ====== Model 2 ======= : 一組一組用 CHBF算，每種senario要算 100組 再平均
                
                # 將 job 按照 cost (workload) 排序

                list_c = instance[2][p]
                list_b = instance[3][p]

                list_cb = list(zip(list_c, list_b))        # [3,5]  [8,1]  -> [(3,8),(5,1)]
                s_cb    = sorted(list_cb, reverse = True)  # [(3,8),(5,1)] -> [(5,1),(3,8)]
                o_cb    = list(zip(*s_cb))                 # [(5,1),(3,8)] -> [(5,3),(1,8)]

                list_c  = list(o_cb[0])                    # [5,3]
                list_b  = list(o_cb[1])                    # [1,8]

                mach = []                 # machine
                ca = instance[4][p]

                for r in range(num_machine):

                    mach.append([])    # machine set中安排一台台 machine 被 append 進來
                    mach[r].append(0)  # 初始的 benefit和 是 0
                    mach[r].append(ca) # 初始的 capacity  是 ca

                TotalBnft_2 = 0             
                for s in range(num_job):

                    mach = sorted(mach)                           # 按照 benefit和 由小而大排序
                    # print(machine)

                    for r in range(num_machine):                  # 由最小 benefit和 的先開始
                        
                        if list_c[s] <= mach[r][1]:               # machine r 還有 capacity 餘裕
                            
                            #print(mach[r])
                            mach[r][1] -= list_c[s]
                            mach[r][0] += list_b[s]
                            #print(mach[r])
 
                            break
    
                for r in range(num_machine):                  
                    TotalBnft_2 += mach[r][0]                           # 將所有 machine 的 benefit和加起來成為一組
                    #print(mach[r][0])
                    
                TotalBenefit_2 = np.append(TotalBenefit_2, TotalBnft_2) # 將 100組全 append在一起 
                #print(TotalBenefit_2)
                
                
            # ====== Model 1 Objective value: ====== #
            
            list_M1           = list(TotalBenefit_1)
            TotalBenefit_100_1 = sum(list_M1) / len(list_M1)
            TotalBenefit_ijk_sn_1.append(TotalBenefit_100_1)   # i*j*k 種 senario 迭代
            
            # ====== Model 2 Objective value: ====== #
            
            list_M2           = list(TotalBenefit_2)
            TotalBenefit_100_2 = sum(list_M2) / len(list_M2)
            TotalBenefit_ijk_sn_2.append(TotalBenefit_100_2)  # i*j*k 種 senario 迭代
            
            print('=========================' + str(i) + str(j) + str(k) + '=========================') # 分隔           
            
print(TotalBenefit_ijk_sn_1) # list
print(TotalBenefit_ijk_sn_2) # list

TotalBenefit_ijk_sn_ratio = list(map(lambda a, b: a / b, TotalBenefit_ijk_sn_2, TotalBenefit_ijk_sn_1))
TotalBenefit_ijk_sn_ratio

CPLEX 12.8.0.0: optimal integer solution; objective 560
120 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 525
4841 MIP simplex iterations
1169 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 458
1593 MIP simplex iterations
370 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 495
4815 MIP simplex iterations
1147 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 495
115 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 681
619 MIP simplex iterations
116 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 599
568 MIP simplex iterations
94 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 444
15352 MIP simplex iterations
5087 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 510
139 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.

CPLEX 12.8.0.0: optimal integer solution; objective 502
135 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 526
26859 MIP simplex iterations
5027 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 509
569 MIP simplex iterations
115 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 485
2640 MIP simplex iterations
459 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 530
7351 MIP simplex iterations
1620 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 510
13847 MIP simplex iterations
1858 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 535
6032 MIP simplex iterations
1524 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 515
659 MIP simplex iterations
114 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 555
639 MIP simplex iterations
104 branch-and-bound nodes
C

CPLEX 12.8.0.0: optimal integer solution; objective 13875
524 MIP simplex iterations
146 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 16533
138 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 14030
3866 MIP simplex iterations
1257 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 18038
7976 MIP simplex iterations
2252 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 19217
607 MIP simplex iterations
180 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 15951
1347 MIP simplex iterations
478 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution within mipgap or absmipgap; objective 19426
2025 MIP simplex iterations
631 branch-and-bound nodes
absmipgap = 1.2, relmipgap = 6.17729e-05
CPLEX 12.8.0.0: optimal integer solution; objective 14821
72 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer sol

CPLEX 12.8.0.0: optimal integer solution; objective 405
88 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 325
1235 MIP simplex iterations
279 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 415
8484 MIP simplex iterations
1174 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 368
5763 MIP simplex iterations
1338 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 344
8086 MIP simplex iterations
1887 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 450
1718 MIP simplex iterations
574 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 330
76 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 350
50 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 400
10611 MIP simplex iterations
2468 branch-and-bound nodes
CPLEX 12

CPLEX 12.8.0.0: optimal integer solution; objective 96.33118598
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 94.15580093
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 90.48960742
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 82.84268082
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 75.51483062
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 91.48533454
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 90.94927795
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 99.28230669
0 MIP simplex iterations
0 branch-and-bound nodes
CPLEX 12.8.0.0: optimal integer solution; objective 94.69418989
0 MIP simplex iterations

[0.96368873742291472,
 0.96065582047715459,
 0.98366305140870469,
 0.94426570212425798,
 0.96558246090732314,
 0.85714799251590179,
 0.98243196050455783,
 0.69456256610615841,
 1.0,
 1.0,
 1.0,
 1.0]

In [99]:
TotalBenefit_ijk_sn_ratio = list(map(lambda a, b: a / b, TotalBenefit_ijk_sn_2, TotalBenefit_ijk_sn_1))
TotalBenefit_ijk_sn_ratio

[0.96368873742291472,
 0.96065582047715459,
 0.98366305140870469,
 0.94426570212425798,
 0.96558246090732314,
 0.85714799251590179,
 0.98243196050455783,
 0.69456256610615841,
 1.0,
 1.0,
 1.0,
 1.0]

## Other Function

In [22]:
# Increase the costs of beef and ham

cost = ampl.getParameter('cost')
cost.setValues({'BEEF': 5.01, 'HAM': 4.55})
print("Increased costs of beef and ham.")

# 再解一次

ampl.solve()
print("New objective value:", totalcost.value())

# 印出其中項目

Buy = ampl.getVariable('Buy')
print("Buy['BEEF'].val = {}".format(Buy['BEEF'].value()))

# Display the dual value of each constraint

diet = ampl.getConstraint('diet')
for nutr in nutrients:
    print("diet['{}'].dual = {}".format(nutr, diet[nutr].dual()))

Increased costs of beef and ham.
CPLEX 12.8.0.0: optimal solution; objective 144.0120033
0 simplex iterations (0 in phase I)
New objective value: 144.01200332502074
Buy['BEEF'].val = 5.226932668329172
diet['A'].dual = 0.0
diet['C'].dual = 0.0
diet['B1'].dual = 0.0
diet['B2'].dual = 0.7999285120532
diet['NA'].dual = -0.007531172069825434
diet['CAL'].dual = 0.0
